In [1]:
import pandas as pd
from tulip.data.bloomberg import BloombergClient as bb
from tulip.plots import plot_histograms

EPS = {
    "USA - SPX": "SPX Index",
    "USA - Nasdaq": "NDX Index",
    "USA - SPX EW": "SPW Index",
    "USA - Ex-Mag 7": "B500XM7T Index",
    "USA - Mag 7": "BM7T Index",
    "USA - R2K": "RTY Index",
    "EUR - Stoxx 50": "SX5E Index",
    "EUR - Stoxx 600": "SXXP Index",
    "EUR - DAX": "DAX Index",
    "JPN - Nikkei": "NKY Index",
    "JPN - Topix": "TPX Index",
    "CHN - CSI 300": "SHSZ300 Index",
    "CHN - Hang Seng": "HSI Index",
    "CHN - Hang Seng Tech": "HSTECH Index",
}

FIELDS_OF_INTEREST = [
    "BEST_SALES",
    "TRAIL_12M_SALES_PER_SH",
    "BEST_EBITDA",
    "TRAIL_12M_EBITDA_PER_SHARE",
    "BEST_EPS",
    "TRAIL_12M_EPS",
]

data = {}
for s, eps_ticker in EPS.items():
    data[s] = bb.bdh(eps_ticker, fields=FIELDS_OF_INTEREST).droplevel(0, axis=1)

### Earnings Per Share
#### Summary

In [2]:
eps_analysis = {k: dict() for k in data.keys()}

for s, v in data.items():
    # Get data for the selected market
    eps_analysis[s]["ts"] = ts = v[FIELDS_OF_INTEREST].ffill().resample("QE").last()
    eps_analysis[s]["ann_chg"] = ann_chg = eps_analysis[s]["ts"].pct_change(4)
    eps_analysis[s]["expectations"] = expectations = pd.DataFrame(
        index=eps_analysis[s]["ts"].index,
        data={
            "EXP_EPS": eps_analysis[s]["ts"]["BEST_EPS"]
            / eps_analysis[s]["ts"]["TRAIL_12M_EPS"]
            - 1,
            "EXP_EBITDA": eps_analysis[s]["ts"]["BEST_EBITDA"]
            / eps_analysis[s]["ts"]["TRAIL_12M_EBITDA_PER_SHARE"]
            - 1,
            "EXP_SALES": eps_analysis[s]["ts"]["BEST_SALES"]
            / eps_analysis[s]["ts"]["TRAIL_12M_SALES_PER_SH"]
            - 1,
        },
    )

    eps_analysis[s]["sales_yoy"] = sales_yoy = (
        ann_chg["TRAIL_12M_SALES_PER_SH"].dropna().rename("Sales YoY") * 100
    )
    eps_analysis[s]["sales_yoy_expectation"] = sales_yoy_expectation = (
        expectations["EXP_SALES"].dropna().iloc[-1] * 100
    )

    eps_analysis[s]["ebitda_yoy"] = ebitda_yoy = (
        ann_chg["TRAIL_12M_EBITDA_PER_SHARE"].dropna().rename("EBITDA YoY") * 100
    )
    eps_analysis[s]["ebitda_yoy_expectation"] = ebitda_yoy_expectation = (
        expectations["EXP_EBITDA"].dropna().iloc[-1] * 100
    )

    eps_analysis[s]["eps_yoy"] = eps_yoy = (
        ann_chg["TRAIL_12M_EPS"].dropna().rename("EPS Actual YoY") * 100
    )
    eps_analysis[s]["eps_yoy_expectation"] = eps_yoy_expectation = (
        expectations["EXP_EPS"].dropna().iloc[-1] * 100
    )

    eps_analysis[s]["eps_yoy_expectations"] = eps_yoy_expectations = (
        expectations["EXP_EPS"].dropna().rename("EPS Expected YoY") * 100
    )

    eps_analysis[s]["histogram_chart"] = plot_histograms(
        [sales_yoy, ebitda_yoy, eps_yoy, eps_yoy_expectations],
        thresholds=[
            sales_yoy_expectation,
            ebitda_yoy_expectation,
            eps_yoy_expectation,
            eps_yoy_expectation,
        ],
        titles=["Sales YoY", "EBITDA YoY", "EPS Actual YoY", "EPS Expected YoY"],
        title=f"<b>{s}</b>",
        xlabel="YoY Change (%)",
        layout="grid",
    )


In [3]:
for s, v in eps_analysis.items():
    v["histogram_chart"].show()

In [4]:
from IPython.display import Markdown

Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:39_